**Q1. What are the most common reasons for missing data in ETL pipelines?**

- Data entry error or incomplete user froms
- System failuser during data capture
- Data intregration issue from multipal sources
- Optional failds left blank by user
- Data curruption during extraction or transformation


**Q2. Why is blindly deleting rows with missing values considered a bad practice in ETL?**

- It cause data loss
- Can introduce bias in analysis
- Reduces deta set size and accuracy
- Missing values may carry importnt business insights


**Q3. Explain the difference between: Listwise deletion, Column deletion**

- Listwise deletion removes entire rows containing missing values, while column deletion removes
entire columns with excessive missing data. Listwise deletion is used when missing data is minimal,
whereas column deletion is suitable when a column is irrelevant


**Q4. Why is median imputation preferred over mean imputation for skewed data such as income?**

- Meen is affeected by ouliers
- Median represent central tendancy better
- Income deta is usually right-skewed
- Median provide more stable results


**Q5. What is forward fill and in what type of dataset is it most useful?**

- Forverd fills replace missing values with the previous vailed value
- Useful in time-series
- Example: sales, stock price data


**Q6. Why should flagging missing values be done before imputation in an ETL workflow?**

- Helps track original missing data
- Allowes better modle interpretation
- Prevent loss of information after imputation


**Q7. Consider a scenario where income is missing for many customers.**

- Indicates customers unwilling to share income
- Shows low data quality regions
- Helps identify privacy-sensitive segments
- Useful for marketing strategy decisions



In [ ]:

-- Step 1: Create Database

CREATE DATABASE etl_missing_data;
USE etl_missing_data;

-- Step 2: Create Table

CREATE TABLE customer_data (
    Customer_ID INT PRIMARY KEY,
    Name VARCHAR(50),
    City VARCHAR(50),
    Monthly_Sales INT,
    Income INT,
    Region VARCHAR(20)
);

-- Step 3: Insert Data
INSERT INTO customer_data VALUES
(101, 'Rahul Mehta', 'Mumbai', 12000, 65000, 'West'),
(102, 'Anjali Rao', 'Bengaluru', NULL, NULL, 'South'),
(103, 'Suresh Iyer', 'Chennai', 15000, 72000, 'South'),
(104, 'Neha Singh', 'Delhi', NULL, NULL, 'North'),
(105, 'Amit Verma', 'Pune', 18000, 58000, NULL),
(106, 'Karan Shah', 'Ahmedabad', NULL, 61000, 'West'),
(107, 'Pooja Das', 'Kolkata', 14000, NULL, 'East'),
(108, 'Riya Kapoor', 'Jaipur', 16000, 69000, 'North');

-- View Original Data

SELECT * FROM customer_data;

-- =========================================
-- Q8: Listwise Deletion
-- Remove rows where Region is missing
-- =========================================

-- Identify affected rows

SELECT * FROM customer_data
WHERE Region IS NULL;

-- Delete rows with missing Region

DELETE FROM customer_data
WHERE Region IS NULL;

-- View data after deletion

SELECT * FROM customer_data;

-- =========================================
-- Q9: Forward Fill for Monthly_Sales
-- =========================================

UPDATE customer_data
SET Monthly_Sales = (
    SELECT Monthly_Sales
    FROM (
        SELECT Customer_ID, Monthly_Sales
        FROM customer_data
    ) AS temp
    WHERE temp.Customer_ID < customer_data.Customer_ID
      AND temp.Monthly_Sales IS NOT NULL
    ORDER BY temp.Customer_ID DESC
    LIMIT 1
)
WHERE Monthly_Sales IS NULL;

-- View data after forward fill
SELECT * FROM customer_data;

-- =========================================
-- Q10: Flagging Missing Income
-- =========================================

-- Add flag column
ALTER TABLE customer_data
ADD COLUMN Income_Missing_Flag INT;

-- Update flag values
UPDATE customer_data
SET Income_Missing_Flag =
CASE
    WHEN Income IS NULL THEN 1
    ELSE 0
END;

-- View final dataset
SELECT * FROM customer_data;

-- Count customers with missing income
SELECT COUNT(*) AS Missing_Income_Count
FROM customer_data
WHERE Income_Missing_Flag = 1;
